In [1]:
import pymongo
con_str = "localhost:27017"
db = pymongo.MongoClient(con_str, serverSelectionTimeoutMS = 10000)
try:
    # print(db.server_info())
    
    # 查询数据库中所有数据库的名称
    db_names = db.list_database_names()
    print(db_names)
except Exception:
    print("connect failed!")

['admin', 'config', 'db_Weibo_SH0331', 'db_Weibo_SH_0331_All', 'db_tvcgR3', 'db_weiboDB', 'local', 'test', 'weiboInfoAll']


In [7]:
targetDB = 'db_Weibo_SH0331'
targetCol = 'col_weibo_timePeriod_0'
if targetDB in db_names:
    db_Weibo_SH0331 = db['db_Weibo_SH0331']
    col_names = db_Weibo_SH0331.list_collection_names()
    print(col_names)
    if targetCol in col_names:
        col_weibo_timePeriod_0 = db_Weibo_SH0331['col_weibo_timePeriod_0']
        print(col_weibo_timePeriod_0.count_documents({}))

['col_weibo_timePeriod_16', 'col_weibo_timePeriod_23', 'col_weibo_timePeriod_17', 'col_weibo_timePeriod_9', 'col_weibo_timePeriod_18', 'col_weibo_timePeriod_15', 'col_weibo_timePeriod_1', 'col_weibo_timePeriod_13', 'col_weibo_timePeriod_3', 'col_Weibo_SH0331', 'col_weibo_timePeriod_21', 'col_weibo_timePeriod_14', 'col_weibo_timePeriod_20', 'col_weibo_timePeriod_12', 'col_weibo_timePeriod_10', 'col_weibo_timePeriod_22', 'col_weibo_timePeriod_4', 'col_weibo_timePeriod_7', 'col_weibo_timePeriod_19', 'col_weibo_timePeriod_8', 'col_weibo_timePeriod_5', 'col_weibo_timePeriod_2', 'col_weibo_timePeriod_6', 'col_weibo_timePeriod_0', 'col_weibo_timePeriod_11']
69160


## 目标：自动在集合 `col_weibo_timePeriod_0 `~ `col_weibo_timePeriod_23`内，新建自定义的字段
* 事实上，已经在程序中`DP01`中指出，**不存在某个字段的时候，不需要新建，直接操作该字段（比如，给一个不存在的字段赋值）就会自动新建该字段。**
* 但是因为后期还是需要用到自动化，所以本次整理程序的时候，就重点回顾和记录一下函数`exec()`的学习的时候，最重要的点：`locals()`<br>
\>\_<就因为它搞了半天，以此记之！

### `exec()` 知识点记录 ★★★★☆
* 正常的语句1（不在函数内部）<br><br>

```python
col_weibo_timePeriod_9 = db_Weibo_SH0331['col_weibo_timePeriod_9']  # 连接数据库中的集合
nums1 = col_weibo_timePeriod_9.count_documents({})  # nums1 = 81770

exists1 = col_weibo_timePeriod_9.find_one({'titleCutting': {'exists': True}})  # 当前col_weibo_timePeriod9里不存在'titleCutting'字段
print(exists1)  # None
```

* 自动化语句2（在函数内部，批量执行）<br><br>

```python
def makeNewFields(targetCol, newField):
    print(targetCol)  # 查看参数是否传递进来
    print(newField)  # 查看参数是否传递进来

    # 存储了局部变量的一个字典（key-value，键值对）
    loc = locals()  # loc指向的对象一直在变，所以loc也会变（即，在不同位置打印loc时，出现的内容不一样）

    # ● 注意，这里虽然已经连接，但是是在函数内部进行连接的，在函数外部无法访问targetCol对应的字符串指向的集合，即'col_weibo_timePeriod_9'这个集合
    exec(targetCol + " = db_Weibo_SH0331['" + targetCol+ "']")  # exec()函数：执行【连接数据库中的集合】的语句
    exec("nums2 = " + targetCol + ".count_documents({})")
    print(loc)  # 输出loc，可以看到当前的局部变量，此时没有'exists2'这个key

    exec("exists2 = " + targetCol + ".find_one({'" + newField + "': {'exists': True}})")
    print(loc)  # 输出loc的位置不同，对应的loc内容不一样，此时已经有了'exists2'这个key


    '''需要注意：下面这条语句，是在打印全局变量exists2, 如果全局中没有定义，则会报错'''
    # print(exists2)  # NameError: name 'exists2' is not defined     

    '''如果想打印当前函数内部的exists2这个变量对应的值'''
    print(loc['exists2'])  # 不存在，就输出None；存在，就是打印第一条记录(find_one()执行结果)的内容
    print(loc['nums2'])


if __name__ == '__main__':
    targetCol = 'col_weibo_timePeriod_9'  # 随便挑一个collection来试试
    newField = 'titleCutting'
    makeNewFields(targetCol, newField)  
    # col_weibo_timePeriod_9.count_documents({})  # ● NameError: name 'col_weibo_timePeriod_9' is not defined
 
```

In [8]:
col_weibo_timePeriod_9 = db_Weibo_SH0331['col_weibo_timePeriod_9']  # 连接数据库中的集合
nums1 = col_weibo_timePeriod_9.count_documents({})  # nums1 = 81770
exists1 = col_weibo_timePeriod_9.find_one({'titleCutting': {'$exists': True}})  # 当前col_weibo_timePeriod9里不存在'titleCutting'字段
print(exists1)  # None

None


In [17]:
def makeNewFields(targetCol, newField):
    print(targetCol)  # 查看参数是否传递进来
    print(newField)  # 查看参数是否传递进来

    # 存储了局部变量的一个字典（key-value，键值对）
    loc = locals()  # loc指向的对象一直在变，所以loc也会变（即，在不同位置打印loc时，出现的内容不一样）

    # ● 注意，这里虽然已经连接，但是是在函数内部进行连接的，在函数外部无法访问targetCol对应的字符串指向的集合，即'col_weibo_timePeriod_10'这个集合
    exec(targetCol + " = db_Weibo_SH0331['" + targetCol+ "']")  # exec()函数：执行【连接数据库中的集合】的语句
    exec("nums2 = " + targetCol + ".count_documents({})")
    print(loc)  # 输出loc，可以看到当前的局部变量，此时没有'exists2'这个key

    exec("exists2 = " + targetCol + ".find_one({'" + newField + "': {'$exists': True}})")
    print(loc)  # 输出loc的位置不同，对应的loc内容不一样，此时已经有了'exists2'这个key


    '''需要注意：下面这条语句，是在打印全局变量exists2, 如果全局中没有定义，则会报错，可以解除注释，打印试试（一定报错—_—）'''
    # print(exists2)  # NameError: name 'exists2' is not defined     

    '''如果想打印当前函数内部的exists2这个变量对应的值'''
    print(loc['exists2'])  # 不存在，就输出None；存在，就是打印第一条记录(find_one()执行结果)的内容
    print(loc['nums2'])
                                                           

if __name__ == '__main__':
    targetCol = 'col_weibo_timePeriod_10'  # 随便挑一个collection来试试
    newField = 'titleCutting'
    makeNewFields(targetCol, newField)  
    
    # ● NameError: name 'col_weibo_timePeriod_10' is not defined
    # print(col_weibo_timePeriod_10.count_documents({}))  # 报错原因，函数内部连接了，处理了，但是没传到外部。

col_weibo_timePeriod_10
titleCutting
{'targetCol': 'col_weibo_timePeriod_10', 'newField': 'titleCutting', 'loc': {...}, 'col_weibo_timePeriod_10': Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=10000), 'db_Weibo_SH0331'), 'col_weibo_timePeriod_10'), 'nums2': 75401}
{'targetCol': 'col_weibo_timePeriod_10', 'newField': 'titleCutting', 'loc': {...}, 'col_weibo_timePeriod_10': Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=10000), 'db_Weibo_SH0331'), 'col_weibo_timePeriod_10'), 'nums2': 75401, 'exists2': None}
None
75401
